1. <font color=royalblue>Workbook defines a trinomial function which can be used  to evaluate Vanilla and Single Barrier Option</font>
2. <font color=royalblue>Support both American & European Style</font>
3. <font color=royalblue>Supported barrier pricing:</font>
    1. **UP-OUT**
    2. **UP-IN**
    3. **DOWN-OUT**
    4. **DOWN-IN**
4.<font color=royalblue>Note:Does not yet support dividend payments and rebates. Will be incoporated in future versions</font>

In [1]:
import ipywidgets as widgets
from IPython.display import display,clear_output

In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix


In [3]:
#!jupyter serverextension enable voila --sys-prefix

In [4]:
import numpy as np
import math


In [5]:
np.set_printoptions(precision=3,suppress=True)


In [6]:
#Function Starts here:
def trinomialtreepricing(S0,strike,sigma,T,N,r,OptionFlag,ExcerciseType,Barrier,BarrierType,H):
    t= T/N
    
    #Size of UpMove (u) and DownMove (d)
    u = math.exp(sigma*math.sqrt(2*t))
    d = 1/u
    
    #Discount Factor
    df = math.exp(-r*t)
    
    #Probability of UpMove(p_u),DownMove (p_d) & sideways move (p_m)
    component_1 = math.exp(r*t/2)
    component_2 = math.exp(-sigma*math.sqrt(t/2))
    
    denominator_1 = math.exp(sigma*math.sqrt(t/2))
    denominator_2 = math.exp(-sigma*math.sqrt(t/2))
    
    p_u = math.pow((component_1 - component_2)/(denominator_1 - denominator_2),2)
    p_d = math.pow((denominator_1 - component_1)/(denominator_1 - denominator_2),2)
    p_m = 1- p_u - p_d
    
    #empty array to create stock price tree
    S = np.zeros((N*2+1,N+1))
    midpoint = int((N*2+1)/2)
    
    S[midpoint] = S0
    
    #For Loop to populate the price tree S in two tranches
    #The upper tranche or half of the pricing tree
    for i in range(midpoint-1,-1,-1):
        for j in range(1,N+1):
            S[i,j] = S[i+1,j-1]*u
    
    #The lower tranche or half of the pricing tree
    for i in range(midpoint+1,N*2+1):
        for j in range(1,N+1):
            S[i,j] = S[i-1,j-1]*d

    
    ##empty array to create option payoff
    P = np.zeros((N*2+1,N+1))
    
    #For Loop to fill the option payoff array P
    #for i in range(0,S.shape[0]):
    for j in range(N,-1,-1):
        #for j in range(0,S.shape[1]):
        for i in range(len(P) - (j+N)-1,j+N+1):
            #print(i,j)
            if Barrier == 'Y':
                if (BarrierType == 'UP-OUT' and S[i,j] >= H):
                    P[i,j] == 0   
                elif (BarrierType == 'UP-IN' and S[i,j] < H):
                        P[i,j] == 0
                elif (BarrierType == 'DOWN-OUT' and S[i,j] <= H):
                    P[i,j] == 0
                elif (BarrierType == 'DOWN-IN' and S[i,j] > H):
                    P[i,j] == 0 
                elif OptionFlag == 'C':
                    P[i,j] = max(S[i,j] - strike,0)
                elif OptionFlag == 'P':
                    P[i,j] = max(strike - S[i,j] ,0)
                else:
                    0
            elif Barrier == 'N':
                if OptionFlag == 'C':
                    P[i,j] = max(S[i,j] - strike,0)
                
                elif OptionFlag == 'P':
                    P[i,j] = max(strike - S[i,j] ,0)
                else:
                    0
            
    #empty array of option price
    option_price = np.zeros((N*2+1,N+1))
    
    #Discounted recursive option price base on Barrier Condition and whether excercise type is American or European
    for j in range(N,-1,-1):
        if  j == N:
            option_price[:,j] = P[:,j] 
        else:
            for i in range(len(option_price) - (j+N)-1,j+N+1):
                if Barrier == 'Y':
                    if (BarrierType == 'UP-OUT' and S[i,j] >= H):
                        0
                    elif (BarrierType == 'UP-IN' and S[i,j] < H):
                        0
                    elif (BarrierType == 'DOWN-OUT' and S[i,j] <= H):
                        0
                    elif (BarrierType == 'DOWN-IN' and S[i,j] >= H):
                        0   
                    else:
                        if ExcerciseType == 'A': 
                            option_price[i,j] = max((option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df,P[i,j])
                        elif ExcerciseType == 'E':
                            option_price[i,j] = (option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df
                elif Barrier == 'N':
                     if ExcerciseType == 'A': 
                            option_price[i,j] = max((option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df,P[i,j])
                     elif ExcerciseType == 'E':
                        option_price[i,j] = (option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df

    #Just fuff to prettify the print output and return the option price, probabilities and size of up and downmove
    if N > 9:
        print('\033[1m'+ 'For visibility nodes restricted to first 9 time steps'+ "\n" \
              '\033[94m' + '\033[1m' + 'Price Tree'+ "\n" \
              + '\033[1;36m'  + '\033[0m' + str(S[N-9+1:N+9,:9]))
        
        print('\033[1m' + 'For visibility nodes restricted to first 9 time steps'+ "\n" \
              + '\033[33m' + 'Option Price Tree'+ "\n" \
              + '\033[0m' + str(option_price[N-9+1:N+9,:9]) + "\n")

    else:
        print('\033[94m' + '\033[1m' + 'Price Tree'+ "\n" + '\033[0m' + str(S))
        print('\033[1;36m'  + '\033[33m' + 'Option Price Tree'+ "\n" + '\033[0m' + str(option_price) + "\n")

    #return('Option Price: ' + str(np.round(option_price[midpoint,0],3)),\
           #'P(U): ' + str(np.round(p_u,3)),'P(M): ' + str(np.round(p_m,3)),'P(D): ' + str(np.round(p_d,3)),\
          #'Upmove: ' + str(np.round(u,3)),'Downmove: ' + str(np.round(d,3)))
    
    return(print('\033[1m' + '\033[31m' + 'Option Price: ' + '\033[0m' + str(np.round(option_price[midpoint,0],3))+ "\n"\
          +'\033[1m' + '\033[31m' + 'P(U): ' + '\033[0m' + str(np.round(p_u,3))+ "\n"\
          +'\033[1m' + '\033[31m' + 'P(M): ' + '\033[0m' + str(np.round(p_m,3))+ "\n"\
          +'\033[1m' + '\033[31m' + 'P(D): ' + '\033[0m' + str(np.round(p_d,3))+ "\n"\
          +'\033[1m' + '\033[31m' + 'Upmove: ' + '\033[0m' + str(np.round(u,3))+ "\n"\
          + '\033[1m' + '\033[31m' + 'Downmove: ' + '\033[0m' + str(np.round(d,3))))

###Function Ends here

**<font color=royalblue><u>User Input Section</u></font>**

<font color=royalblue><u>The function has 11 inputs:</u></font>
-  S0 -> Initial Price
-  strike -> strike price
-  sigma -> underlying volatility
-  T -> time to expiry in years
-  N -> number of steps for the tree
-  r -> interest rate
-  OptionFlag -> 'C' (Call)/ 'P' (Put)
-  ExcerciseType -> 'A' (American)/ 'P' (European)
- Barrier -> 'Y' (Yes)/ 'N' (No)
-  BarrierType -> 'UP-OUT'/'UP-IN'/'DOWN-OUT'/'DOWN-IN' (**Note: Only required if barrier flag is 'Y'**)
-  H -> Barrier Price (**Note: Only required if barrier flag is 'Y'**)


In [7]:
S0 = widgets.FloatText(value=100,description='S0:',disabled=False)

sigma = widgets.FloatText(value=0.40,description='sigma:',disabled=False)

T = widgets.FloatText(value=1,description='T:',disabled=False)

N = widgets.IntText(value=5,description='N:',disabled=False)

strike = widgets.FloatText(value=60,description='strike:',disabled=False)

r = widgets.FloatText(value=0.05,description='r:',disabled=False)

OptionFlag = widgets.Dropdown(options=['C','P'],description='Option Flag:',disabled=False)

ExcerciseType = widgets.Dropdown(options=['E','A'],description='ExcerciseType: ',disabled=False)

Barrier = widgets.Dropdown(options=['N','Y'],description='Barrier:',disabled=False)

#Only required if it Barrier flag is 'Y'
BarrierType = widgets.Dropdown(options=['None','UP-OUT','UP-IN','DOWN-OUT','DOWN-IN'],description='BarrierType:',disabled=False)
H = widgets.FloatText(value=None,description='H:',disabled=False)

out = widgets.Output()

display(S0,sigma,T,N,strike,r,OptionFlag,ExcerciseType,Barrier,BarrierType,H)


FloatText(value=100.0, description='S0:')

FloatText(value=0.4, description='sigma:')

FloatText(value=1.0, description='T:')

IntText(value=5, description='N:')

FloatText(value=60.0, description='strike:')

...

Dropdown(description='Option Flag:', options=('C', 'P'), value='C')

Dropdown(description='ExcerciseType: ', options=('E', 'A'), value='E')

Dropdown(description='Barrier:', options=('N', 'Y'), value='N')

Dropdown(description='BarrierType:', options=('None', 'UP-OUT', 'UP-IN', 'DOWN-OUT', 'DOWN-IN'), value='None')

FloatText(value=0.0, description='H:')

**<font color=royalblue><u>The function produces 3 outputs:</u></font>**
  
1. Price Tree of the underlying 
2. Option Price Tree using backward induction
3. List of:
    1. Option Price,
    2. P(U) - Probability of Upmove, 
    3. P(M) - Probability of horizontal move, 
    4. P(D) - Probability of down move , 
    5. Upmove, 
    6. Downmove


In [8]:
button = widgets.Button(description='Calculate',color='lime')
out = widgets.Output()

def on_button_clicked(_):
    with out:
        clear_output()
        trinomialtreepricing(S0.value,strike.value,sigma.value,T.value,N.value,r.value,OptionFlag.value,\
                                   ExcerciseType.value,Barrier.value,BarrierType,H)
        

button.on_click(on_button_clicked)

widgets.VBox([button,out])

In [1]:
pip list --format=freeze -> requirements.txt

Note: you may need to restart the kernel to use updated packages.
